In [1]:
import pandas as pd
import json
import matplotlib as plt
import data_module as dm


In [2]:
df = pd.read_csv('data/test_public.csv')

***Clean up data***
Remove examples missing data in polyline and figure out how many examples have NaNs

In [4]:
print(len(df[df['MISSING_DATA'] == True]))

10


In [23]:
df_tu = df_tu[df_tu['MISSING_DATA'] == False]
print(df_tu.head)
print(len(df_tu[df_tu['MISSING_DATA'] == True]))

<bound method NDFrame.head of                      TRIP_ID CALL_TYPE  ORIGIN_CALL  ORIGIN_STAND  TAXI_ID  \
0        1372636858620000589         C          NaN           NaN        0   
1        1372637303620000596         B          NaN           7.0        1   
2        1372636951620000320         C          NaN           NaN        2   
3        1372636854620000520         C          NaN           NaN        3   
4        1372637091620000337         C          NaN           NaN        4   
...                      ...       ...          ...           ...      ...   
1710665  1404171463620000698         C          NaN           NaN      440   
1710666  1404171367620000670         C          NaN           NaN      441   
1710667  1388745716620000264         C          NaN           NaN      439   
1710668  1404141826620000248         B          NaN          12.0      443   
1710669  1404157147620000079         B          NaN          34.0      444   

          TIMESTAMP DAY_TYPE  MIS

In [12]:
for col in df.columns:
    print('NaNs in column %s %i' % (col, len(df[df[col].isna() == True])))

NaNs in column TRIP_ID 0
NaNs in column CALL_TYPE 0
NaNs in column ORIGIN_CALL 1345891
NaNs in column ORIGIN_STAND 904084
NaNs in column TAXI_ID 0
NaNs in column TIMESTAMP 0
NaNs in column DAY_TYPE 0
NaNs in column MISSING_DATA 0
NaNs in column POLYLINE 0
NaNs in column LEN 0
NaNs in column YR 0
NaNs in column MON 0
NaNs in column DAY 0
NaNs in column HR 0
NaNs in column WK 0


Based on the above we probably will need to ignore the ORIGIN_CALL and ORIGIN_STAND columns most of their data is null.

In [19]:
df = df.drop(['ORIGIN_CALL', 'ORIGIN_STAND'], axis=1)
print(df.columns)

KeyError: "['ORIGIN_CALL', 'ORIGIN_STAND'] not found in axis"

We're left with the columns above. Now to see if there are any distinct correlations between columns and travel time. First we'll calculate travel time and add it to a new column. Travel time is calculated using len(POLYLINE[i]) - 1 * 15

In [10]:
def app(val):
    return (len(json.loads(val)) - 1) * 15

In [12]:
df['TRAVEL_TIME'] = df['POLYLINE'].map(app)

In [10]:
df_tu.columns

NameError: name 'df' is not defined

Save updated csv

In [9]:
df.to_csv('data/test_public.csv', index=False)

In [2]:
df_tu = pd.read_csv('data/train_up.csv')


In [58]:
df = df_tu[df_tu['COORDS'] != '[]']

In [59]:
print(df.columns)

Index(['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID',
       'TIMESTAMP', 'DAY_TYPE', 'POLYLINE', 'TRAVEL_TIME', 'COORDS'],
      dtype='object')


In [55]:
df_tu['COORDS'].iloc[1710657]

'[]'

In [40]:
def map_json(val):
    return json.loads(val)

def map_normalize(val):
    normal = []
    for v in val:
        normal.append((v - 4502.893653516295) / 215.0430544729272)
        
    return normal
t = map_normalize(df_tu['COORDS'].map(map_json).tolist())

TypeError: unsupported operand type(s) for -: 'list' and 'float'

In [21]:
print(t[7])
print(df_tu['COORDS'].iloc[0])

[4656.96477760585, 4656.928017844394, 4656.807500005271, 4656.848863636868, 4656.846744060687, 4656.874659513318, 4657.012906604327, 4657.020708493081, 4657.05238827631, 4657.076848902042, 4657.079803476187, 4657.116592256822, 4657.250284859582, 4657.257309756821, 4657.3222717573935, 4657.505342295106, 4657.688561545411, 4657.903187019163, 4658.032432478856, 4658.030989513225, 4658.068346158823, 4658.224459452486, 4658.370350418425, 4658.538974408166, 4658.564966222808, 4658.649340474245, 4658.765727001558, 4658.765896191977, 4658.765090187741, 4658.764284183788, 4658.763308989032, 4658.763308989032, 4658.764114992924, 4658.764582616328]
[4656.468213451219, 4656.4616055530005, 4656.618829905977, 4656.7945874043735, 4656.888901052368, 4656.984081875508, 4656.988343925256, 4657.097616116268, 4657.330327124793, 4657.452747715531, 4657.652907635357, 4657.727042405734, 4657.730306550668, 4657.828214289824, 4657.8448798548425, 4657.861837010968, 4657.918507026974, 4657.983508772181, 4658.120

In [25]:
print(df_tu.columns)

Index(['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID',
       'TIMESTAMP', 'DAY_TYPE', 'POLYLINE', 'TRAVEL_TIME'],
      dtype='object')


In [24]:
df_tu = df_tu.drop(['MISSING_DATA'], axis=1)

In [15]:
df_tu['TAXI_ID'].iloc[523423]

376

In [4]:
t_id = {}
id_ctr = 0

for i in range(len(df)):
    taxi_id = df['TAXI_ID'].iloc[i]
    try:
        t_id[taxi_id]
    except:
        t_id[taxi_id] = id_ctr
        id_ctr += 1

In [8]:
print(list(t_id.keys()))

[7, 8, 9, 10, 11, 12, 1, 2, 3, 4, 5, 6]


In [6]:
def map_id(val):
    return t_id[val]

df['TAXI_ID'] = df['TAXI_ID'].map(map_id)

In [8]:
df['TAXI_ID'].min()

0

In [21]:
len(df[df['TRAVEL_TIME'] <= 0]) / len(df)

0.0

Remove dataset examples that don't contain any gps coordinates or are empty

In [8]:
df = df[df['TRAVEL_TIME'] < 0]
print(df.head)

<bound method NDFrame.head of                      TRIP_ID CALL_TYPE   TAXI_ID   TIMESTAMP DAY_TYPE  \
0        1372636858620000589         C  20000589  1372636858        A   
1        1372637303620000596         B  20000596  1372637303        A   
2        1372636951620000320         C  20000320  1372636951        A   
3        1372636854620000520         C  20000520  1372636854        A   
4        1372637091620000337         C  20000337  1372637091        A   
...                      ...       ...       ...         ...      ...   
1710653  1388660427620000585         C  20000585  1388660427        A   
1710655  1404171463620000698         C  20000698  1404171463        A   
1710656  1404171367620000670         C  20000670  1404171367        A   
1710658  1404141826620000248         B  20000248  1404141826        A   
1710659  1404157147620000079         B  20000079  1404157147        A   

                                                  POLYLINE  TRAVEL_TIME  \
0        [[-8.6186

In [17]:
from datetime import datetime

# Over every single 
def polyline_to_trip_duration(polyline):
  return max(polyline.count("[") - 2, 0) * 15

# This code creates a new column, "LEN", in our dataframe. The value is
# the (polyline_length - 1) * 15, where polyline_length = count("[") - 1
df_tu = df_tu.drop(['LEN'], axis=1)
df_tu["TRAVEL_TIME"] = df_tu["POLYLINE"].apply(polyline_to_trip_duration)



# Because we are assigning multiple values at a time, we need to "expand" our computed (year, month, day, hour, weekday) tuples on 
# the column axis, or axis 1
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
#df[["YR", "MON", "DAY", "HR", "WK"]] = df[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

In [4]:
def parse_time(x):
  # We are using python's builtin datetime library
  # https://docs.python.org/3/library/datetime.html#datetime.date.fromtimestamp

  # Each x is essentially a 1 row, 1 column pandas Series
  dt = datetime.fromtimestamp(x["TIMESTAMP"])
  return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

In [10]:
mean, std = df["LEN"].mean(), df["LEN"].std()
median = df["LEN"].median()

df_trimmed = df[df["LEN"] < mean + 3 * std]

In [11]:
print(len(df_trimmed))

1692763


In [14]:
print('Start at stand %i' % (len(df[df['ORIGIN_STAND'].isna() == False])))

Start at stand 806576


In [37]:
t = {}
t['hi']

KeyError: 'hi'

In [33]:
import math
from math import radians

coord_list = []
unique = {}

def get_coords(coords):
    coords = coords.strip('][').split(', ')
    for c in coords:
        if (c == ''):
            continue

        c = math.floor(float(c))
        try:
            unique[c]
        except Exception:
            unique[c] = 0
        
        unique[c] += 1
            
for i in range(len(df_tu)):
    if i % 50000 == 0:
        print(i)
    
    get_coords(df_tu['COORDS'].iloc[i])

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000
950000
1000000
1050000
1100000
1150000
1200000
1250000
1300000
1350000
1400000
1450000
1500000
1550000
1600000
1650000
1700000


In [34]:
coord_list = list(unique.keys())

In [35]:
rounded = [int(math.ceil(x / 250.0)) * 250 for x in coord_list]

In [38]:
import numpy as np
c_mean = np.mean(coord_list)
c_std = np.std(coord_list)
print(c_mean)
print(c_std)

4502.893653516295
215.0430544729272


In [62]:
t_df = pd.DataFrame(coord_list, columns=['COORD'])

In [72]:
len(t_df[t_df['COORD'] > c_mean * 2 + c_std]) / len(t_df)

0.0

In [73]:
print(c_mean)
print(c_std)

12085.850987173944
4294.907703951505


In [93]:
def map_normalize(val):
    normal = []
    for v in val:
        normal.append((v - 12085.850987173944) / 4294.907703951505)
        
    return normal

def map_json(val):
    return json.loads(val)


In [94]:
t = df['COORDS'].iloc[0]

In [95]:
t = map_json(t)

In [96]:
t = map_normalize(t)

In [97]:
t

[1.348950746228855,
 1.348999907095406,
 1.3474813699466301,
 1.3457356967877072,
 1.344977479960874,
 1.3444111710904871,
 1.3445106088944065,
 1.343799913576223,
 1.3415075503348777,
 1.3397890518650934,
 1.3369427710559652,
 1.3358521212417755,
 1.3358160529443908,
 1.3345400074393805,
 1.3343137570938368,
 1.3341215138304083,
 1.333883399868352,
 1.3334123953873986,
 1.3314733574499045,
 1.3314528199834113,
 1.3314408395691948,
 1.331464800332944,
 1.3314766828143865]

In [1]:
import pandas as pd
import json
import matplotlib as plt
import data_module as dm
from datetime import datetime

In [2]:
train_dl, test_dl, train_len, test_len = dm.get_loader()